In [ ]:


import astropy
print('astropy',astropy.__version__ )
from spectral_cube import SpectralCube      # This is a handy package for working with 3D data cubes
from astropy.coordinates import SkyCoord
from reproject import reproject_interp      
from reproject.mosaicking import find_optimal_celestial_wcs 
import regions
import reproject
print('reproject',reproject.__version__)
import spectral_cube
print('spectral_cube',spectral_cube.__version__)
import numpy as np                          
import pylab                                
import matplotlib 
import matplotlib.gridspec as gridspec                                                                                             
import scipy
import astropy.io.fits as fits                                                          
from astropy.wcs import WCS                 
from astropy import units as u              
import pyspeckit as psk   
import astrodendro
import matplotlib.colors as colors
from matplotlib import pyplot as plt
# Suppress warnings we don't care about:
import sys
if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")
%matplotlib inline
from astropy.convolution import Gaussian1DKernel
#%matplotlib widget

In [ ]:

files=["CMZ_3mm_HCO+_mJy.pbcor.fits","CMZ_3mm_HCN_mJy.pbcor.fits" ,"CMZ_3mm_H13CN_1-0_mJy.pbcor.fits","CMZ_3mm_H13CO+_mJy.pbcor.fits"]

for i,file in enumerate(files):
    
    sc = SpectralCube.read(file).unmasked_copy()    
    #header = fits.getheader(file)
    if i==0:                                # Find location of brightest emission
        z,xp,yp = pylab.where(sc.hdu.data==np.nanmax(sc.hdu.data)) 
        yRaxDec = sc.wcs[:][:][0].pixel_to_world(yp,xp)
    y, x = sc.wcs[:][:][0].world_to_pixel(yRaxDec)# Gets the pixel for the same Ra/dec of each cube
    #Need to reduce cube size, but it won't work if theyre all the same size sometimes?
    
    subcube = sc[:,:,:] # Make a subcube
    
    
    #Put in the right system
    sc_kms = subcube.with_spectral_unit(u.km/u.s,velocity_convention="radio") # Change units from Hz to km/s
    sc_slab = sc_kms.spectral_slab(-1000. *u.km / u.s, 5000. *u.km / u.s)  # Crop out velocities we don't care about    
    sc_slab.allow_huge_operations=True    
    sc_K_kms = sc_slab#.to(u.K)
    print(sc_K_kms.header["RESTFRQ"],sc_K_kms.header)
    #print(sc_K_kms.wcs)
    #print(sc_K_kms.world[:,0,0][0])
    #print(header)
    if i==0:
        vel = np.arange(-250,250,2.5)*u.km/u.s
        
        HCO_Plus = sc_K_kms
        HC0_reproject_interp = HCO_Plus.spectral_interpolate(spectral_grid=vel) # Match velocities too     
        HC0_reproject_interp.write('cmz_HCOp.cube.mJy-kms.pbcor.fits',overwrite=True) 
        del HCO_Plus
    if i==1:
        HCN = sc_K_kms.unmasked_copy()

del sc_kms #rem from mem
del sc_slab #rem from mem
del sc_K_kms #rem from mem
del subcube #rem from mem
del sc #rem from mem

HCN.allow_huge_operations=True

HCOPH = HC0_reproject_interp.header


del HC0_reproject_interp #rem from mem


HCN_reproject = HCN.reproject(HCOPH,use_memmap=True)  
del HCN #rem from mem
del HCOPH
fwhm_factor = np.sqrt(8*np.log(2))

HCN_reprojectGauss = HCN_reproject.spectral_smooth(Gaussian1DKernel(4/fwhm_factor))#I don't know what this is. "This can be useful if you want to interpolate onto a coarser grid but maintain Nyquist sampling. You can then use the spectral_interpolate method to regrid your smoothed spectrum onto a new grid.""
del HCN_reproject
HCN_reproject_interp = HCN_reproject.spectral_interpolate(spectral_grid=vel) # Match velocities too     

del HCN_reprojectGauss

HCN_reproject_interp.write('cmz_HCN.cube.reproject.mJy-kms.pbcor.fits',overwrite=True)
del HCN_reproject_interp         
print("done")        

In [ ]:
files =['cmz_HCOp.cube.mJy-kms.pbcor.fits',"CMZ_3mm_HCN_mJy.pbcor.fits" ,"CMZ_3mm_H13CN_1-0_mJy.pbcor.fits","CMZ_3mm_H13CO+_mJy.pbcor.fits"]


for i,file in enumerate(files):
    
    sc = SpectralCube.read(file).unmasked_copy()    
    #header = fits.getheader(file)
    if i==0:                                # Find location of brightest emission
        z,xp,yp = pylab.where(sc.hdu.data==np.nanmax(sc.hdu.data)) 
        yRaxDec = sc.wcs[:][:][0].pixel_to_world(yp,xp)
    y, x = sc.wcs[:][:][0].world_to_pixel(yRaxDec)# Gets the pixel for the same Ra/dec of each cube
    #Need to reduce cube size, but it won't work if theyre all the same size sometimes?
    
    subcube = sc[:,:,:] # Make a subcube
    
    
    #Put in the right system
    sc_kms = subcube.with_spectral_unit(u.km/u.s,velocity_convention="radio") # Change units from Hz to km/s
    sc_slab = sc_kms.spectral_slab(-1000. *u.km / u.s, 5000. *u.km / u.s)  # Crop out velocities we don't care about    
    sc_slab.allow_huge_operations=True    
    sc_K_kms = sc_slab#.to(u.K)
    
    #print(sc_K_kms.wcs)
    #print(sc_K_kms.world[:,0,0][0])
    #print(header)
    if i==0:
        vel = np.arange(-250,250,2.5)*u.km/u.s
        
        HCO_Plus = sc_K_kms.unmasked_copy()
        #HCO_Plus.write('cmz_HCN.cube.reproject.K-kms.pbcor.fits',overwrite=True)
        
    if i==2:
        
        H13CN = sc_K_kms.unmasked_copy()

del sc_kms #rem from mem
del sc_slab #rem from mem
del sc_K_kms #rem from mem
del subcube #rem from mem
del sc #rem from mem

#del scW
H13CN.allow_huge_operations=True

HCOPH = HCO_Plus.header


del HCO_Plus #rem from mem


H13CN_reproject = H13CN.reproject(HCOPH,use_memmap=True)  
del H13CN #rem from mem
del HCOPH
fwhm_factor = np.sqrt(8*np.log(2))

H13CN_reprojectGauss = H13CN_reproject.spectral_smooth(Gaussian1DKernel(4/fwhm_factor))#I don't know what this is. "This can be useful if you want to interpolate onto a coarser grid but maintain Nyquist sampling. You can then use the spectral_interpolate method to regrid your smoothed spectrum onto a new grid.""
del H13CN_reproject
H13CN_reproject_interp = H13CN_reprojectGauss.spectral_interpolate(spectral_grid=vel) # Match velocities too     
del H13CN_reprojectGauss
H13CN_reproject_interp.write('cmz_H13CN.cube.reproject.mJy-kms.pbcor.fits',overwrite=True)        
del H13CN_reproject_interp         
print("done")        

In [5]:
files =['cmz_HCOp.cube.mJy-kms.pbcor.fits',"CMZ_3mm_HCN_mJy.pbcor.fits" ,"CMZ_3mm_H13CN_1-0_mJy.pbcor.fits","CMZ_3mm_H13CO+_mJy.pbcor.fits"]


for i,file in enumerate(files):
    
    sc = SpectralCube.read(file).unmasked_copy()    
    #header = fits.getheader(file)
    if i==0:                                # Find location of brightest emission
        z,xp,yp = pylab.where(sc.hdu.data==np.nanmax(sc.hdu.data)) 
        yRaxDec = sc.wcs[:][:][0].pixel_to_world(yp,xp)
    y, x = sc.wcs[:][:][0].world_to_pixel(yRaxDec)# Gets the pixel for the same Ra/dec of each cube
    #Need to reduce cube size, but it won't work if theyre all the same size sometimes?
    
    subcube = sc[:,:,:] # Make a subcube
    
    
    #Put in the right system
    sc_kms = subcube.with_spectral_unit(u.km/u.s,velocity_convention="radio") # Change units from Hz to km/s
    sc_slab = sc_kms.spectral_slab(-1000. *u.km / u.s, 5000. *u.km / u.s)  # Crop out velocities we don't care about    
    sc_slab.allow_huge_operations=True    
    sc_K_kms = sc_slab#.to(u.K)
    
    #print(sc_K_kms.wcs)
    #print(sc_K_kms.world[:,0,0][0])
    #print(header)
    if i==0:
        vel = np.arange(-250,250,2.5)*u.km/u.s
        
        HCO_Plus = sc_K_kms.unmasked_copy()
        #HCO_Plus.write('cmz_HCN.cube.reproject.K-kms.pbcor.fits',overwrite=True)
        
    if i==3:
        
        H13COp= sc_K_kms.unmasked_copy()

del sc_kms #rem from mem
del sc_slab #rem from mem
del sc_K_kms #rem from mem
del subcube #rem from mem
del sc #rem from mem

#del scW
H13COp.allow_huge_operations=True

HCOPH = HCO_Plus.header


del HCO_Plus #rem from mem


H13COp_reproject = H13COp.reproject(HCOPH,use_memmap=True)  
del H13COp #rem from mem
del HCOPH
fwhm_factor = np.sqrt(8*np.log(2))

H13COp_reprojectGauss = H13COp_reproject.spectral_smooth(Gaussian1DKernel(4/fwhm_factor))#I don't know what this is. "This can be useful if you want to interpolate onto a coarser grid but maintain Nyquist sampling. You can then use the spectral_interpolate method to regrid your smoothed spectrum onto a new grid.""
del H13COp_reproject
H13COp_reproject_interp = H13COp_reprojectGauss.spectral_interpolate(spectral_grid=vel) # Match velocities too     
del H13COp_reprojectGauss
H13COp_reproject_interp.write('cmz_H13COp.cube.reproject.mJy-kms.pbcor.fits',overwrite=True)  

del H13COp_reproject_interp         
print("done")        

done


In [13]:


files = pylab.array(['cmz_HCOp.cube.mJy-kms.pbcor.fits','gc_ATLASGAL_850micron_dust.fits'])
      



for i,file in enumerate(files):
    
    if i==0:
        subcube = SpectralCube.read(file) 
    
    
    if i==1:
        subcube = spectral_cube.Projection.from_hdu(fits.open(file)[0])
    #print(subcube.hdu.header)
    
    
    #Put in the right system
    
    subcube.allow_huge_operations=True
    
    
    
    if i==0:
        vel, dec, ra = subcube.world[:,0,0]            # Grab the velocity axis to use for rebinning
        
        NGC_HCO_Plus = subcube.moment0()
    if i==1:
        Cont = subcube
        

        
del dec
del ra
Cont.allow_huge_operations=True
#HCOPH = NGC_HCO_Plus.header
HCOPH = NGC_HCO_Plus.hdu.header
#HCOPH['NAXIS'] = 2
print(HCOPH)
#print([HCOPH['NAXIS{0}'.format(i + 1)] for i in range(HCOPH['NAXIS'])][::-1])
print([Cont.hdu.header['NAXIS{0}'.format(i + 1)] for i in range(Cont.hdu.header['NAXIS'])][::-1])
print(np.shape(NGC_HCO_Plus))
del NGC_HCO_Plus #rem from mem

Cont_reproject =Cont.reproject(HCOPH)  
del HCOPH

Cont_reproject.write('CMZ_Continuum_Reproject.fits',overwrite=True) 
print(np.shape(Cont_reproject))
del Cont_reproject          
print("done")        

SIMPLE  =                    T / conforms to FITS standard                      BITPIX  =                  -32 / array data type                                NAXIS   =                    2 / number of array dimensions                     NAXIS1  =                  765                                                  NAXIS2  =                  165                                                  BLANK   =                   -1  /                                               BUNIT   = 'beam-1 m mJy s-1'                                                    CELLSCAL= 'CONSTANT'  /                                                         RESTFREQ=    8.91885260000E+10  /                                               BMAJ    =    1.08300000429E-02  /                                               BMIN    =    1.08300000429E-02  /                                               DATAMIN =   -1.53687677383E+01  /                                               DATAMAX =    3.78722572327E+00  /       

In [14]:

#Left Half of the CMZ
files = pylab.array(['cmz_HCOp.cube.mJy-kms.pbcor.fits',"ADP.2016-01-20T15:35:07.077.fits"])
      



for i,file in enumerate(files):
    
    if i==0:
        subcube = SpectralCube.read(file) 
    
    
    if i==1:
        subcube = spectral_cube.Projection.from_hdu(fits.open(file)[0])
    #print(subcube.hdu.header)
    
    
    #Put in the right system
    
    subcube.allow_huge_operations=True
    
    
    
    if i==0:
        vel, dec, ra = subcube.world[:,0,0]            # Grab the velocity axis to use for rebinning
        
        NGC_HCO_Plus = subcube.moment0()
    if i==1:
        Cont = subcube
        

        
del dec
del ra
Cont.allow_huge_operations=True
#HCOPH = NGC_HCO_Plus.header
HCOPH = NGC_HCO_Plus.hdu.header
#HCOPH['NAXIS'] = 2
print(HCOPH)
#print([HCOPH['NAXIS{0}'.format(i + 1)] for i in range(HCOPH['NAXIS'])][::-1])
print([Cont.hdu.header['NAXIS{0}'.format(i + 1)] for i in range(Cont.hdu.header['NAXIS'])][::-1])
print(np.shape(NGC_HCO_Plus))
del NGC_HCO_Plus #rem from mem

Cont_reproject =Cont.reproject(HCOPH)  

del HCOPH

Cont_reproject.write('CMZ_Continuum_Reproject_Left_Half.fits',overwrite=True) 
print(np.shape(Cont_reproject))
del Cont_reproject          
print("done")        

SIMPLE  =                    T / conforms to FITS standard                      BITPIX  =                  -32 / array data type                                NAXIS   =                    2 / number of array dimensions                     NAXIS1  =                  765                                                  NAXIS2  =                  165                                                  BLANK   =                   -1  /                                               BUNIT   = 'beam-1 m mJy s-1'                                                    CELLSCAL= 'CONSTANT'  /                                                         RESTFREQ=    8.91885260000E+10  /                                               BMAJ    =    1.08300000429E-02  /                                               BMIN    =    1.08300000429E-02  /                                               DATAMIN =   -1.53687677383E+01  /                                               DATAMAX =    3.78722572327E+00  /       

In [19]:
path = 'CMZ_CO_3_2_Right.cube.reproject.K-kms.pbcor.fits'

sc = SpectralCube.read(path)
print(sc.header)

SIMPLE  =                    T / conforms to FITS standard                      BITPIX  =                  -64 / array data type                                NAXIS   =                    3                                                  NAXIS1  =                  765                                                  NAXIS2  =                  165                                                  NAXIS3  =                  200                                                  COMMENT   FITS (Flexible Image Transport System) format is defined in 'AstronomyCOMMENT   and Astrophysics', volume 376, page 359; bibcode: 2001A&A...376..359H LBOUND1 =                 -524 / Pixel origin along axis 1                      LBOUND2 =                 -124 / Pixel origin along axis 2                      LBOUND3 =                 -366 / Pixel origin along axis 3                      LABEL   = 'T%s60+%v30+A%^50+%<20+*%+   corrected antenna temperature' / Label ofBUNIT   = 'K       '           / Units o

In [18]:
#Left
files = pylab.array(["CMZ_3mm_HCO+_mJy.pbcor.fits","12CO_GC_001-002_mosaic.fit"])


for i,file in enumerate(files):
    
    sc = SpectralCube.read(file).unmasked_copy()    
    #header = fits.getheader(file)
    if i==0:                                # Find location of brightest emission
        z,xp,yp = pylab.where(sc.hdu.data==np.nanmax(sc.hdu.data)) 
        yRaxDec = sc.wcs[:][:][0].pixel_to_world(yp,xp)
    y, x = sc.wcs[:][:][0].world_to_pixel(yRaxDec)# Gets the pixel for the same Ra/dec of each cube
    #Need to reduce cube size, but it won't work if theyre all the same size sometimes?
    
    subcube = sc[:,:,:] # Make a subcube
    
    if i==1:
        lll=0
        #subcube=subcube*u.K #The CO obs are not in units, but unit K is stated in the header
    #Put in the right system
    sc_kms = subcube.with_spectral_unit(u.km/u.s,velocity_convention="radio") # Change units from Hz to km/s
    sc_slab = sc_kms.spectral_slab(-1000. *u.km / u.s, 5000. *u.km / u.s)  # Crop out velocities we don't care about    
    sc_slab.allow_huge_operations=True 
    #print(sc_kms[0][0])
    sc_K_kms = sc_slab#.to(u.K)
    
    #print(sc_K_kms.wcs)
    #print(sc_K_kms.world[:,0,0][0])
    #print(header)
    vel = np.arange(-250,250,2.5)*u.km/u.s
    #print(vel)
    if i==0:
        vel = np.arange(-250,250,2.5)*u.km/u.s
        
        HCO_Plus = sc_K_kms.unmasked_copy()
        HCO_Plus.write('cmz_HCOp.cube.K-kms.pbcor.fits',overwrite=True) 
        
    if i==1:
        CO32 = sc_K_kms.unmasked_copy()

del sc_kms #rem from mem
del sc_slab #rem from mem
del sc_K_kms #rem from mem
del subcube #rem from mem
del sc #rem from mem

CO32.allow_huge_operations=True

HCOPH = HCO_Plus.header
HCOPH['CTYPE3'] = 'VRAD'#Change from VRAD-W2F to VRAD, since it means sampled in wave, expressed in freq, same as VRAD, see: https://docs.astropy.org/en/stable/_modules/astropy/wcs/wcsapi/fitswcs.html?highlight=VRAD#

del HCO_Plus #rem from mem


CO32_reproject = CO32.reproject(HCOPH,use_memmap=True)  
del CO32 #rem from mem
del HCOPH

fwhm_factor = np.sqrt(8*np.log(2))

CO32_reprojectGauss = CO32_reproject.spectral_smooth(Gaussian1DKernel(4/fwhm_factor))#I don't know what this is. "This can be useful if you want to interpolate onto a coarser grid but maintain Nyquist sampling. You can then use the spectral_interpolate method to regrid your smoothed spectrum onto a new grid.""
del CO32_reproject

CO32_reproject_interp = CO32_reprojectGauss.spectral_interpolate(spectral_grid=vel) # Match velocities to -250 250 range 
print(CO32_reproject_interp.world[:][0][0],CO32_reprojectGauss.world[:][0][0])
del CO32_reprojectGauss
CO32_reproject_interp = CO32_reproject_interp.hdu
CO32_reproject_interp = SpectralCube.read(CO32_reproject_interp)
CO32_reproject_interp.write('CMZ_CO_3_2_Left.cube.reproject.K-kms.pbcor.fits',overwrite=True)
del CO32_reproject_interp         
print("done")        

[[-250. -250. -250. ... -250. -250. -250.]
 [-250. -250. -250. ... -250. -250. -250.]
 [-250. -250. -250. ... -250. -250. -250.]
 ...
 [-250. -250. -250. ... -250. -250. -250.]
 [-250. -250. -250. ... -250. -250. -250.]
 [-250. -250. -250. ... -250. -250. -250.]] km / s [[-299.5793098 -299.5793098 -299.5793098 ... -299.5793098 -299.5793098
  -299.5793098]
 [-299.5793098 -299.5793098 -299.5793098 ... -299.5793098 -299.5793098
  -299.5793098]
 [-299.5793098 -299.5793098 -299.5793098 ... -299.5793098 -299.5793098
  -299.5793098]
 ...
 [-299.5793098 -299.5793098 -299.5793098 ... -299.5793098 -299.5793098
  -299.5793098]
 [-299.5793098 -299.5793098 -299.5793098 ... -299.5793098 -299.5793098
  -299.5793098]
 [-299.5793098 -299.5793098 -299.5793098 ... -299.5793098 -299.5793098
  -299.5793098]] km / s
done
